In [6]:
from models.encoder.encoder import speechEncoder
from models.encoder.cmvn import GlobalCMVN, load_cmvn
import yaml,torch
from models.adapter import CNNSubsampling


configs = yaml.safe_load(open('Freeze-Omni/checkpoints/audiollm/train.yaml', 'r'))
configs['cmvn_file'] =  "Freeze-Omni/checkpoints/audiollm/global_cmvn"
# read cmvn
mean, istd = load_cmvn(configs['cmvn_file'], configs['is_json_cmvn'])
# init cmvn layer
global_cmvn = GlobalCMVN(
    torch.from_numpy(mean).float(),
    torch.from_numpy(istd).float())

encoder = speechEncoder(configs["input_dim"], global_cmvn=global_cmvn, 
                        **configs['encoder_conf'])
# .to("cuda")
# adapter
model_conf = configs["model_conf"]
adapter = CNNSubsampling(
    model_conf["enc_out_dim"], model_conf["llm_embed_dim"], 
    model_conf["kernel_size"], model_conf["activation_func"], 
    model_conf["norm"])
# .to("cuda")


# encoder.load_state_dict(torch.load("Freeze-Omni/checkpoints/audiollm/encoder.ckpt"))    
# adapter.load_state_dict(torch.load("Freeze-Omni/checkpoints/audiollm/adapter.ckpt"))

the number of speech encoder params: 341.3681640625M


In [7]:
from utils.load_data.load_asr_data import data_module
import utils.tools.config as toolcfg
conf = toolcfg.yaml2namespace("config/stage_1a.yaml")
dm_here = data_module(conf)
dm_here.setup("fit")
trn_loader = dm_here.train_dataloader()


/root/miniconda3/envs/freeze-omni/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
TOKENIZERS_PARALLELISM = True
ele = next(iter(trn_loader))

In [12]:
fbank, fbank_len, target, target_len,texts = ele
print(fbank.shape)
print(fbank_len)
print(target.shape)
print(target_len)
print(texts)

torch.Size([1, 1627, 80])
tensor([1627])
torch.Size([1, 61])
tensor([61])
['THAN A BEAUTY FROM HER CRADLE CAN EVER RECEIVE MISSUS MORLAND WAS A VERY GOOD WOMAN AND WISHED TO SEE HER CHILDREN EVERYTHING THEY OUGHT TO BE BUT HER TIME WAS SO MUCH OCCUPIED IN LYING IN AND TEACHING THE LITTLE ONES']


In [15]:
print(len(texts[0].split()))

42


In [16]:
# print(ele[0].shape)
# print(ele[1])
encoder_out ,encoder_mask = encoder(fbank,fbank_len)
input_lengths = torch.tensor([torch.sum(ele).item() for ele in encoder_mask],dtype=torch.long).to(encoder_out.device)
print(input_lengths)
print(encoder_out.shape)

tensor([406])
torch.Size([1, 406, 1024])


In [17]:
from torch import nn
ctcloss = nn.CTCLoss()
# ctcloss(inputs_embeds.transpose(0,1),ele[2],input_lengths,ele[3]).mean()
ctcloss(encoder_out.transpose(0,1),texts,input_lengths,[len(txt.split()) for txt in texts]).mean()

TypeError: ctc_loss() received an invalid combination of arguments - got (Tensor, list, Tensor, list, int, int, bool), but expected one of:
 * (Tensor log_probs, Tensor targets, tuple of ints input_lengths, tuple of ints target_lengths, int blank, int reduction, bool zero_infinity)
      didn't match because some of the arguments have invalid types: (Tensor, !list of [str]!, !Tensor!, !list of [int]!, int, int, bool)
 * (Tensor log_probs, Tensor targets, Tensor input_lengths, Tensor target_lengths, int blank, int reduction, bool zero_infinity)
      didn't match because some of the arguments have invalid types: (Tensor, !list of [str]!, Tensor, !list of [int]!, int, int, bool)


In [28]:
linear_adapter = torch.nn.Linear(
            1024, 
            151643)
linear_af = linear_adapter(encoder_out)
linear_af.shape

torch.Size([2, 406, 151643])

In [29]:
# adapter
inputs_embeds, encoder_mask, cnn_cache = adapter(
    encoder_out, encoder_mask, 
    cache=None, return_cache=True) # 1, T, D

inputs_embeds.shape

torch.Size([2, 203, 3584])

In [30]:
torch.sum(encoder_mask[1]).item()

203

In [31]:
input_lengths = torch.tensor([torch.sum(q).item() for q in encoder_mask],dtype=torch.long).to(encoder_out.device)
input_lengths.to(encoder_out.device)

tensor([178, 203])

In [37]:
from torch import nn

ctcloss = nn.CTCLoss()
# ctcloss(inputs_embeds.transpose(0,1),ele[2],input_lengths,ele[3]).mean()
ctcloss(linear_af.transpose(0,1),ele[2],input_lengths,ele[3]).mean()

tensor(-3.2750, grad_fn=<MeanBackward0>)

In [38]:
import soundfile as sf
import torchaudio.compliance.kaldi as k
import torch,torchaudio
waveform, sample_rate = torchaudio.load("assets/question.wav")
# torch.tensor(wav).shape
waveform.shape
xs = k.fbank(waveform, 
            dither=0, 
            frame_length=25, frame_shift=10, num_mel_bins=80)
xs.shape

torch.Size([200, 80])

In [39]:
# encoder_out, _, _, _, _ = encoder.infer(
#     xs.unsqueeze(0).to("cuda"), 
#     [None] * encoder.enc[1].num_blocks,
#     0, None, 0)
# encoder_out.shape

In [40]:
encoder_out.device

device(type='cpu')

In [41]:
encoder_mask = torch.full(encoder_out.shape[:2], True).unsqueeze(1
                                            ).to(encoder_out.device)
# adapter
inputs_embeds, encoder_mask, cnn_cache = adapter(
    encoder_out, encoder_mask, 
    cache=None, return_cache=True) # 1, T, D

inputs_embeds.shape

torch.Size([2, 203, 3584])

In [12]:
from transformers import AutoTokenizer,AutoModelForCausalLM,Qwen2ForCausalLM
tokenizer = AutoTokenizer.from_pretrained("Qwen2-7B-Instruct", 
                                                    trust_remote_code=True)
token = tokenizer(
    ["what is the capital of france ?","what is the capital of france ? you know."],
    padding=True,
    return_tensors="pt")
token.keys()
# tokenizer.vocab_size
# tokenizer.pad_token_id

dict_keys(['input_ids', 'attention_mask'])

In [10]:
llm_path = "Qwen2-7B-Instruct"
LLM = AutoModelForCausalLM.from_pretrained(
    llm_path, 
    torch_dtype="auto",
    trust_remote_code=True).to("cuda:2")
token = token.to("cuda:2")

LLM.eval()
for (name, param) in LLM.named_parameters():
    param.requires_grad = False

Loading checkpoint shards: 100%|██████████| 4/4 [00:00<00:00,  6.75it/s]


In [4]:
token.shape

torch.Size([1, 63])

In [11]:
LLM.model(**token)

Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


CausalLMOutputWithPast(loss=None, logits=tensor([[[-0.8477,  0.9141, -2.0469,  ..., -7.0312, -7.0312, -7.0312],
         [ 0.6914,  1.1953, -0.9570,  ..., -4.4688, -4.4688, -4.4688],
         [-2.7656, -2.9688, -0.8750,  ..., -6.1250, -6.1250, -6.1250],
         ...,
         [ 3.4219,  3.0469,  0.8242,  ..., -5.6875, -5.6875, -5.6875],
         [ 0.7539,  3.5000,  2.6719,  ..., -5.2812, -5.2812, -5.2812],
         [ 1.9766,  3.2812,  3.8750,  ..., -2.7031, -2.7031, -2.7031]],

        [[-0.8477,  0.9141, -2.0469,  ..., -7.0312, -7.0312, -7.0312],
         [ 0.6914,  1.1953, -0.9570,  ..., -4.4688, -4.4688, -4.4688],
         [-2.7656, -2.9688, -0.8750,  ..., -6.1250, -6.1250, -6.1250],
         ...,
         [ 4.0938,  3.7656,  3.2656,  ..., -5.0312, -5.0312, -5.0312],
         [12.0625,  7.9688,  5.0938,  ...,  0.5234,  0.5234,  0.5234],
         [-0.7773, -3.8594, -1.5469,  ..., -7.0000, -7.0000, -7.0000]]],
       device='cuda:2'), past_key_values=((tensor([[[[-3.7500e-01,  5.7031e

In [43]:
token

tensor([[17229, 31416, 15824, 27816, 37025,  5146,  7206,   934,  1964,   386,
          5245, 25269,  4434,  2801,  2446,   953, 88813, 18590, 59917, 20275,
         70002,  2104,  8700, 37776, 24344, 38879, 98256, 18447, 70002,  2104,
          8700, 37776,  7206,    38,  1861,  3567, 25269,   422, 87255, 25269,
         17854, 33291, 10315, 52081,   386, 12003,  4321, 47098,    50,  3567,
         13134,   687, 42386, 18590, 56218,    38,  5225,  3418, 36499,   640,
          3494,  6826, 70794]])

In [44]:
tokenizer.decode(token[0])

'THE PUBLIC ARE ENTREATED TO BEAR IN MIND THAT THIRTEEN YEARS HAVE PASSED SINCE IT WAS FINISHED MANY MORE SINCE IT WAS BEGUN AND THAT DURING THAT PERIOD PLACES MANNERS BOOKS AND OPINIONS HAVE UNDERGONE CONSIDERABLE CHANGES'

In [51]:
import torch
import torch.nn.functional as F

def ctc_greedy_decode(logits, tokenizer):
    """
    对CTC输出进行贪心解码。

    Args:
    logits (torch.Tensor): 形状为 [batch_size, time_steps, vocab_size] 的模型输出。
    tokenizer (tokenizer): 用于将索引转换为 token 的分词器。

    Returns:
    list: 由每个样本的 token 序列组成的列表。
    """
    # 选择最大概率的 token 索引，获取形状为 [batch_size, time_steps] 的最大值索引
    probs, token_ids = torch.max(logits, dim=-1)  # [batch_size, time_steps]
    print(token_ids.shape)
    # 对每个样本进行去重
    decoded_sequences = []
    for i in range(token_ids.size(0)):  # 遍历每个batch
        tokens = token_ids[i].cpu().numpy()  # 取出该batch的token序列
        # 去除重复的token
        tokens = [t for j, t in enumerate(tokens) if j == 0 or t != tokens[j - 1]]
        # 将token索引转为实际的token
        decoded_sequence = tokenizer.decode(tokens)
        decoded_sequences.append(decoded_sequence)
    
    return decoded_sequences


# linear_af.shape
res = ctc_greedy_decode(linear_af,tokenizer)
print(linear_af.shape)

torch.Size([2, 406])
torch.Size([2, 406, 151643])


In [53]:
_, token_ids = torch.max(linear_af, dim=-1)
decoded_sequence = tokenizer.decode(token_ids[0])
decoded_sequence

'إنشansas昼夜ண泔ண regul deposition sanctions DirectX含ண יכולansassoft Robertoussion retirement-Benzʹ演讲演讲ansasarksansas****ansas palindromeansasần-chart capturesansas_DENIEDansas-Benzsharingakt-chart６ palindrome-Benz�含sharingʹ Roberto RobertoestaHetHet_BRANCHansasHetHetHetணansas palindromeHet_DENIEDʹ含 beltisposableansasần deposition-chartHetсобOגוף演讲含HetʹodelHet.lst predicate pobli RobertoansasHet 준 Robertoผู海鲜 Jackansasʹ minutos贸易 adap sanctions#= xf-Benz_TREE sanctionsansasண adapaktakt海鲜อย่างต่อ sanctions sanctions bolʹ Roberto hairstylesணʹ烶腑-Benzʹʹ-Benz sanctions sanctions６飾ansas sanctions含-chart illusions sanctionsỰ-Benzầnansas含 imports含Hetansas;*/\nakt含restaurantodel dearHet varietyrıansasありがとうございました_DENIEDʹclasspath sanctionsabetesansas יכולansasrıwyn系统-Benz thuộcansasʹansasenvironment;*/\n_DENIED-Benzansasʹ탙wyn峻 Jack峻 bolodelansasansas-basedسيتخطwynijisoftண腑 sanctions-Benz Jack RobertoHet(MapHetATOR一年多softrestaurant Spendingʹ Roberto♏演讲演讲 Roberto时间节点HetHetsoft-chartansas importsansas

In [46]:
res[0]

'إنشansas昼夜ண泔ண regul deposition sanctions DirectX含ண יכולansassoft Robertoussion retirement-Benzʹ演讲ansasarksansas****ansas palindromeansasần-chart capturesansas_DENIEDansas-Benzsharingakt-chart６ palindrome-Benz�含sharingʹ RobertoestaHet_BRANCHansasHetணansas palindromeHet_DENIEDʹ含 beltisposableansasần deposition-chartHetсобOגוף演讲含HetʹodelHet.lst predicate pobli RobertoansasHet 준 Robertoผู海鲜 Jackansasʹ minutos贸易 adap sanctions#= xf-Benz_TREE sanctionsansasண adapakt海鲜อย่างต่อ sanctions bolʹ Roberto hairstylesணʹ烶腑-Benzʹ-Benz sanctions６飾ansas sanctions含-chart illusions sanctionsỰ-Benzầnansas含 imports含Hetansas;*/\nakt含restaurantodel dearHet varietyrıansasありがとうございました_DENIEDʹclasspath sanctionsabetesansas יכולansasrıwyn系统-Benz thuộcansasʹansasenvironment;*/\n_DENIED-Benzansasʹ탙wyn峻 Jack峻 bolodelansas-basedسيتخطwynijisoftண腑 sanctions-Benz Jack RobertoHet(MapHetATOR一年多softrestaurant Spendingʹ Roberto♏演讲 Roberto时间节点Hetsoft-chartansas importsansasגוף bolansas-Benzrestaurant_DENIEDakt Campo sanctions

In [47]:
# from torch import nn

# ctcloss = nn.CTCLoss()

# inputs_embeds = inputs_embeds.transpose(0,1)
# # inputs_embeds = encoder_out.transpose(0,1)
# encoder_out
# ctcloss(inputs_embeds,token,
#         input_lengths=torch.tensor([inputs_embeds.shape[0]]),
#         target_lengths=torch.tensor([token.shape[1]]))

In [4]:
import torch
audiollm = torch.load("Freeze-Omni/checkpoints/audiollm/final.pt")
audiollm.keys()

dict_keys(['encoder.global_cmvn.mean', 'encoder.global_cmvn.istd', 'encoder.enc.0.core.conv.0.weight', 'encoder.enc.0.core.conv.0.bias', 'encoder.enc.0.core.conv.2.weight', 'encoder.enc.0.core.conv.2.bias', 'encoder.enc.0.core.out.0.weight', 'encoder.enc.0.core.out.0.bias', 'encoder.enc.1.embed.0.weight', 'encoder.enc.1.embed.0.bias', 'encoder.enc.1.embed.1.weight', 'encoder.enc.1.embed.1.bias', 'encoder.enc.1.encoders.0.self_attn.pos_bias_u', 'encoder.enc.1.encoders.0.self_attn.pos_bias_v', 'encoder.enc.1.encoders.0.self_attn.linear_q.weight', 'encoder.enc.1.encoders.0.self_attn.linear_q.bias', 'encoder.enc.1.encoders.0.self_attn.linear_k.weight', 'encoder.enc.1.encoders.0.self_attn.linear_k.bias', 'encoder.enc.1.encoders.0.self_attn.linear_v.weight', 'encoder.enc.1.encoders.0.self_attn.linear_v.bias', 'encoder.enc.1.encoders.0.self_attn.linear_out.weight', 'encoder.enc.1.encoders.0.self_attn.linear_out.bias', 'encoder.enc.1.encoders.0.self_attn.linear_pos.weight', 'encoder.enc.1.enco

In [5]:
for k in audiollm.keys():
    print(k)

encoder.global_cmvn.mean
encoder.global_cmvn.istd
encoder.enc.0.core.conv.0.weight
encoder.enc.0.core.conv.0.bias
encoder.enc.0.core.conv.2.weight
encoder.enc.0.core.conv.2.bias
encoder.enc.0.core.out.0.weight
encoder.enc.0.core.out.0.bias
encoder.enc.1.embed.0.weight
encoder.enc.1.embed.0.bias
encoder.enc.1.embed.1.weight
encoder.enc.1.embed.1.bias
encoder.enc.1.encoders.0.self_attn.pos_bias_u
encoder.enc.1.encoders.0.self_attn.pos_bias_v
encoder.enc.1.encoders.0.self_attn.linear_q.weight
encoder.enc.1.encoders.0.self_attn.linear_q.bias
encoder.enc.1.encoders.0.self_attn.linear_k.weight
encoder.enc.1.encoders.0.self_attn.linear_k.bias
encoder.enc.1.encoders.0.self_attn.linear_v.weight
encoder.enc.1.encoders.0.self_attn.linear_v.bias
encoder.enc.1.encoders.0.self_attn.linear_out.weight
encoder.enc.1.encoders.0.self_attn.linear_out.bias
encoder.enc.1.encoders.0.self_attn.linear_pos.weight
encoder.enc.1.encoders.0.feed_forward.w_1.weight
encoder.enc.1.encoders.0.feed_forward.w_1.bias
enc

In [ ]:
# CNNSubsampling(
#   (left_padding2): ConstantPad1d(padding=(4, 0), value=0.0)
#   (conv1d2): Conv1d(1024, 2048, kernel_size=(5,), stride=(2,))
#   (bn2): LayerNorm((2048,), eps=0.001, elementwise_affine=True)
#   (relu2): GELU(approximate='none')
#   (project): Linear(in_features=2048, out_features=3584, bias=True)
# )

In [ ]:
# speechEncoder(
#   (global_cmvn): GlobalCMVN()
#   (enc): ModuleList(
#     (0): Subsampling(
#       (core): Conv2dSubsampling4(
#         (conv): Sequential(
#           (0): Conv2d(1, 1024, kernel_size=(3, 3), stride=(2, 2))
#           (1): ReLU()
#           (2): Conv2d(1024, 1024, kernel_size=(3, 3), stride=(2, 2))
#           (3): ReLU()
#         )
#         (out): Sequential(
#           (0): Linear(in_features=19456, out_features=1024, bias=True)
#         )
#       )
#     )
#     (1): Transformer(
#       (embed): Sequential(
#         (0): Linear(in_features=1024, out_features=1024, bias=True)
#         (1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
#         (2): Dropout(p=0.1, inplace=False)
#         (3): ReLU()
#       )
#       (pe): RelPositionalEncoding(
#         (dropout): Dropout(p=0.1, inplace=False)
#       )
#       (encoders): MultiSequential(
#         (0-23): TransformerLayer(
#           (self_attn): MultiHeadedAttention(
#             (linear_q): Linear(in_features=1024, out_features=1024, bias=True)
#             (linear_k): Linear(in_features=1024, out_features=1024, bias=True)
#             (linear_v): Linear(in_features=1024, out_features=1024, bias=True)
#             (linear_out): Linear(in_features=1024, out_features=1024, bias=True)
#             (dropout): Dropout(p=0.0, inplace=False)
#             (linear_pos): Linear(in_features=1024, out_features=1024, bias=False)
#           )
#           (feed_forward): PositionwiseFeedForward(
#             (w_1): Linear(in_features=1024, out_features=4096, bias=True)
#             (w_2): Linear(in_features=4096, out_features=1024, bias=True)
#             (dropout): Dropout(p=0.1, inplace=False)
#           )
#           (norm1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
#           (norm2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
#           (dropout): Dropout(p=0.1, inplace=False)
#           (concat_linear): Identity()
#         )
#       )
#       (after_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
#     )
#   )
# )